# Imports

In [1]:
import pickle
import numpy as np
import rmlib

In [2]:
rm = rmlib.RMLib()

Robot Ready


# File Name

In [21]:
file_name = "assembly_features"

# Load/Create Dictionary 

In [23]:
try:
    with open(file_name, 'rb') as f:
        feature_dictionary = pickle.load(f)
        print("File Loaded")
        print('Dictionary Features:')
        for item1 in feature_dictionary:
            print(item1+":")
            for item2 in feature_dictionary[item1]:
                print('      ',item2,' = ',feature_dictionary[item1][item2])
                
except:
    print("No File Found, Creating New Dictionary")
    feature_dictionary = {}

File Loaded
Dictionary Features:
safe:
       pose  =  [-0.2875122767581866, -0.048576704794951284, 0.28681665788637356, -2.2215003746228157, 2.2198375308394622, -0.001894878661018758]
block_1:
       cloud_locator_params  =  [170, 0.0008, 0.0033, 500]
       view_distance  =  0.06
       pose  =  [-0.33210424881795475, 0.09036874254102065, 0.03680491070259806, -2.2215590912943823, 2.2197782547212523, -0.0017003364767290898]


# Create Feature

In [13]:
feature_name = "block_1"
feature_dictionary[feature_name] = {}

# Set Global Location

In [8]:
rm.close_gripper()

0

In [14]:
rm.align_gripper_with_axis()

In [15]:
#Move finger tips to part and save location
global_pose = rm.get_tcp_pose()
feature_dictionary[feature_name]["pose"] = global_pose

In [16]:
#Set view distance
view_height = 0.06
feature_dictionary[feature_name]["view_distance"] = view_height

#Move gripper to view location
rm.movel(np.add(global_pose,[0,0,view_height,0,0,0]))
rm.move_camera_over_tcp()

# Circle Location Parameters

In [ ]:
#Tune center circle parameters
#Tune parameters to minimize false circles and find desired circle in most images
circle_locator_params = rm.tune_circle_locator(min_hough_dist=100, param1=14, param2=13, image_blur=0, z_dist_to_cir=view_height, cir_dia=0.012, dia_tol=0.001, max_loops=5)
print(circle_locator_params)
feature_dictionary[feature_name]["circle_locator_params"] = circle_locator_params

# General Part Location Parameters

In [17]:
rm.open_gripper()

0

In [18]:
#Tune general part parameters
general_part_locator_params = rm.tune_cloud_locator(disparity_shift=170, leaf_size=0.0008, search_radius=0.0033, min_cluster_size=500)
print(general_part_locator_params)
feature_dictionary[feature_name]["cloud_locator_params"] = general_part_locator_params

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.015239958771838001, 0.10255384415235462, …

[170, 0.0008, 0.0033, 500]


# Save Feature

In [22]:
with open(file_name, 'wb') as f:
    pickle.dump(feature_dictionary, f, pickle.DEFAULT_PROTOCOL)